In [7]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, Date,String
from sqlalchemy.orm import Session
import pandas as pd
import numpy as np

In [8]:
measure_df=pd.read_csv("hawaii_measurements.csv")
station_df=pd.read_csv("hawaii_stations.csv")

In [12]:
cleanmeasure_df = measure_df[np.isfinite(measure_df['prcp'])]
cleanmeasure_df.to_csv("clean_measurements.csv", encoding='utf-8')
print(cleanmeasure_df.isnull().sum())

station    0
date       0
prcp       0
tobs       0
dtype: int64


In [22]:
cleanmeasure_df.head(50)

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70
6,USC00519397,2010-01-08,0.00,64
7,USC00519397,2010-01-09,0.00,68
8,USC00519397,2010-01-10,0.00,73
9,USC00519397,2010-01-11,0.01,64
10,USC00519397,2010-01-12,0.00,61


In [18]:
station_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [16]:
Base = declarative_base()
engine = create_engine("sqlite:///Hawaii.sqlite",echo=False)
Base.metadata.create_all(engine)
session = Session(engine)

In [17]:
class Measurement(Base):
    __tablename__="measurement"
    id = Column(Integer, primary_key=True)
    station=Column(String(255))
    latitude=Column(Float)
    longitude=Column(Float)
    elevation=Column(Float)

In [18]:
class Station(Base):
    __tablename__="station"
    id = Column(Integer, primary_key=True)
    station=Column(String(255))
    name=Column(String(255))  
    date=Column(Date)
    prcp=Column(Float)
    tobs=Column(Integer)

In [19]:
cleanmeasure_df.to_sql(con=engine, index_label='id', name=Measurement.__tablename__, if_exists='replace')
station_df.to_sql(con=engine, index_label='id', name=Station.__tablename__, if_exists='replace')


In [20]:
from sqlalchemy import  inspect
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [21]:
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id BIGINT
station TEXT
date TEXT
prcp FLOAT
tobs BIGINT
